## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Countries,Temp,Humidity,Pressure,Latitude,Longitude
0,Severo-Yeniseyskiy,RU,31.10,86,1033,60.3725,93.0408
1,Kavieng,PG,84.94,71,1009,-2.5744,150.7967
2,Bluff,NZ,50.81,77,1030,-46.6000,168.3333
3,Bethel,US,66.88,18,1027,41.3712,-73.4140
4,Lüderitz,NaN,59.02,61,1016,-26.6481,15.1594


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
search_cities_df = (city_data_df.Temp <= max_temp) & (city_data_df.Temp >= min_temp)
preferred_cities_df = city_data_df.loc[search_cities_df]
preferred_cities_df

,City,Countries,Temp,Humidity,Pressure,Latitude,Longitude
1,Kavieng,PG,84.94,71,1009,-2.5744,150.7967
5,Marshfield,US,75.31,78,1016,44.6689,-90.1718
6,Puerto Ayora,EC,73.54,91,1010,-0.7393,-90.3518
8,Tevaitoa,PF,80.89,78,1013,-16.7833,-151.5000
10,Hilo,US,76.91,76,1018,19.7297,-155.0900
...,...,...,...,...,...,...,...
681,Palauig,PH,83.73,65,1009,15.4350,119.9045
682,Clarksburg,US,79.25,19,1023,39.2807,-80.3445
685,Porto Belo,BR,72.12,89,1016,-27.1578,-48.5531
691,Brus Laguna,HN,82.26,75,1011,15.7500,-84.4833


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 1 to 694
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   City       243 non-null    object 
 1   Countries  243 non-null    object 
 2   Temp       243 non-null    float64
 3   Humidity   243 non-null    int64  
 4   Pressure   243 non-null    int64  
 5   Latitude   243 non-null    float64
 6   Longitude  243 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 15.2+ KB


In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(how="any")

,City,Countries,Temp,Humidity,Pressure,Latitude,Longitude
1,Kavieng,PG,84.94,71,1009,-2.5744,150.7967
5,Marshfield,US,75.31,78,1016,44.6689,-90.1718
6,Puerto Ayora,EC,73.54,91,1010,-0.7393,-90.3518
8,Tevaitoa,PF,80.89,78,1013,-16.7833,-151.5000
10,Hilo,US,76.91,76,1018,19.7297,-155.0900
...,...,...,...,...,...,...,...
681,Palauig,PH,83.73,65,1009,15.4350,119.9045
682,Clarksburg,US,79.25,19,1023,39.2807,-80.3445
685,Porto Belo,BR,72.12,89,1016,-27.1578,-48.5531
691,Brus Laguna,HN,82.26,75,1011,15.7500,-84.4833


In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Countries", "Temp", "Humidity", "Latitude", "Longitude"]].copy().reset_index(drop=True)

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df.head(10)

,City,Countries,Temp,Humidity,Latitude,Longitude,Hotel Name,Hotel Address
0,Kavieng,PG,84.94,71,-2.5744,150.7967,,
1,Marshfield,US,75.31,78,44.6689,-90.1718,,
2,Puerto Ayora,EC,73.54,91,-0.7393,-90.3518,,
3,Tevaitoa,PF,80.89,78,-16.7833,-151.5000,,
4,Hilo,US,76.91,76,19.7297,-155.0900,,
5,Goderich,CA,77.32,14,43.7501,-81.7165,,
6,Albany,US,72.00,53,42.6001,-73.9662,,
7,Oddur,SO,74.50,84,4.1213,43.8894,,
8,Fayaoué,NC,77.36,70,-20.6500,166.5333,,
9,Vaini,TO,80.76,65,-21.2000,-175.2000,,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# 6c. Get latitude and longitude from DataFrame
hotel_names = []
hotel_addresses = []

for indx, row in hotel_df.iterrows():
    print(row.City)
    target_coordinates = f"{row.Latitude}, {row.Longitude}"
    
    params["location"] = target_coordinates
    
# 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 6e. Make request and retrieve the JSON data from the search.
    response = requests.get(base_url, params=params)
    
    if (response.status_code == 200):
        places_data = response.json()
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.        
        if (len(places_data["results"]) > 0):
            hotel_names.append(places_data["results"][0]["name"])
            hotel_addresses.append(places_data["results"][0]["vicinity"])
        else:
            hotel_names.append("")
            hotel_addresses.append("")
    else:
        hotel_names.append("")
        hotel_addresses.append("")
        
hotel_df["Hotel Name"] = hotel_names
hotel_df["Hotel Address"] = hotel_addresses
hotel_df.head(10)
        

Kavieng
Marshfield
Puerto Ayora
Tevaitoa
Hilo
Goderich
Albany
Oddur
Fayaoué
Vaini
Merritt Island
Jamestown
Bandarbeyla
Rikitea
Sabá
Takhatpur
Avarua
Quatre Cocos
Biak
Poum
Belmonte
Cockburn Town
Mitú
Waterloo
Bambous Virieux
Hamilton
Celestún
Butaritari
Saint George
San Policarpo
Atuona
Hobyo
Hithadhoo
Matara
Arraial do Cabo
Jiwani
Barinitas
Labuhan
Guánica
El Tocuyo
Funtua
Luganville
Nouadhibou
Manado
Nūzvīd
Isangel
Maceió
Saint-Philippe
Pangoa
Umluj
Gamba
Kanigiri
Kieta
Bengkulu
Tiznit Province
Kapaa
São Filipe
Lorengau
Kahului
Sola
Sal Rei
Manta
Itacarambi
Paracuru
George Town
Pakse
Hong Kong
Faanui
Mahébourg
Dingle
Paka
Polewali
Tezu
Vitorino Freire
Harper
Jilotzingo
Puerto Lleras
Santo Tomas
Robertsport
Duartina
Coruripe
Zabid
Canindé de São Francisco
Puro
Garissa
Caravelas
Souillac
Najrān
Conde
Ixtapa
Basoko
Kisanga
Sechura
Alofi
Naze
Bathsheba
Borazjan
Salalah
Buala
Nísia Floresta
Rodrigues Alves
Ubari
Bandundu Province
Bonthe
Show Low
Meulaboh
Kulhudhuffushi
Moundou
Bourail
Mat

,City,Countries,Temp,Humidity,Latitude,Longitude,Hotel Name,Hotel Address
0,Kavieng,PG,84.94,71,-2.5744,150.7967,Nusa Island Retreat,"Nusa Island Kavieng New Ireland PG 631, Kavieng"
1,Marshfield,US,75.31,78,44.6689,-90.1718,"Holiday Inn Conference Ctr Marshfield, an IHG ...","750 South Central Avenue, Marshfield"
2,Puerto Ayora,EC,73.54,91,-0.7393,-90.3518,Finch Bay Galapagos Hotel,"Barrio Punta Estrada, Puerto Ayora"
3,Tevaitoa,PF,80.89,78,-16.7833,-151.5000,Hôtel Raiatea Lodge,BP 680 Tumaraa Raiatea
4,Hilo,US,76.91,76,19.7297,-155.0900,Hilo Hawaiian Hotel,"71 Banyan Drive, Hilo"
5,Goderich,CA,77.32,14,43.7501,-81.7165,Harmony Inn,"242 Bayfield Road, Goderich"
6,Albany,US,72.00,53,42.6001,-73.9662,,
7,Oddur,SO,74.50,84,4.1213,43.8894,Banjanay Hotel,"4VCQ+JJM, Xuddur"
8,Fayaoué,NC,77.36,70,-20.6500,166.5333,Chez Dydyce Guesthouse,"BP151 Tribu de Banoutr, 98814"
9,Vaini,TO,80.76,65,-21.2000,-175.2000,Keleti Beach Resort,"QQW9+G2M, Ha'ateiho"


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[hotel_df["Hotel Name"] != ""]

,City,Countries,Temp,Humidity,Latitude,Longitude,Hotel Name,Hotel Address
0,Kavieng,PG,84.94,71,-2.5744,150.7967,Nusa Island Retreat,"Nusa Island Kavieng New Ireland PG 631, Kavieng"
1,Marshfield,US,75.31,78,44.6689,-90.1718,"Holiday Inn Conference Ctr Marshfield, an IHG ...","750 South Central Avenue, Marshfield"
2,Puerto Ayora,EC,73.54,91,-0.7393,-90.3518,Finch Bay Galapagos Hotel,"Barrio Punta Estrada, Puerto Ayora"
3,Tevaitoa,PF,80.89,78,-16.7833,-151.5000,Hôtel Raiatea Lodge,BP 680 Tumaraa Raiatea
4,Hilo,US,76.91,76,19.7297,-155.0900,Hilo Hawaiian Hotel,"71 Banyan Drive, Hilo"
...,...,...,...,...,...,...,...,...
237,Ugoofaaru,MV,83.71,78,5.6667,73.0000,Ungoofaaru Inn,"M2CH+8HH, Haveeree Hingun, Ungoofaaru"
238,Palauig,PH,83.73,65,15.4350,119.9045,Alperi Botique Resort,"CWQ5+GWR, Mendoza Street, Palauig"
239,Clarksburg,US,79.25,19,39.2807,-80.3445,Greenbrier Motel & Restaurant,"200 Buckhannon Pike, Clarksburg"
240,Porto Belo,BR,72.12,89,-27.1578,-48.5531,Costa Verde Beach Hotel,"Avenida Leopoldo Zarling, 1612 - Bombas, Bombi..."


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Countries</dt><dd>{Countries}</dd>
and {Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Latitude", "Longitude"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}

coordinates = hotel_df.loc[:, ["Latitude", "Longitude"]].values

popups = []
for indx, row in hotel_df.iterrows():
    info_box_template = f"""
    <dl>
    <dt>Name</dt><dd>{row["Hotel Name"]}</dd>
    <dt>Address</dt><dd>{row["Hotel Address"]}</dd>
    <dt>Countries</dt><dd>{row["Countries"]}</dd>
    </dl>
    """
    popups.append(info_box_template)


fig = gmaps.figure(layout=figure_layout, map_type='HYBRID')

markers = gmaps.marker_layer(coordinates, info_box_content=popups)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))